# Carregando os dados

A base que será utilizada apresenta músicas de diversos gêneros musicais diferentes. Vamos precisar filtrar eles.

In [1]:
import pandas as pd
import re
import os

# Arquivo de entrada
input_file = './corpus-musicas-brasileiras.csv'

# Leitura do CSV, ignorando linhas corrompidas
skip_lines = [ 10002, 10003, 11341, 11342, 19356, 19357, 19364, 19365,
               140164, 140165, 140197, 140198, 182920, 182921, 182953, 182954 ]

df = pd.read_csv( input_file, encoding_errors = 'ignore', skiprows = skip_lines )

# Filtra as linhas que contém "Forró", ignorando maiúsculas/minúsculas e evitando erros com valores nulos
# df = df[df['Gênero Musical'].str.contains('Forró', case=False, na=False)]

# Filtrar gêneros desejados
# generos = ['Forró', 'Piseiro', 'Forró; Piseiro']
# df = df[df['Gênero Musical'].isin(generos)]

# Selecionar colunas relevantes
df = df[ [ 'Nome da Música', 'Letra da Música' ] ]

# Criar pasta para salvar as músicas individualmente
os.makedirs( 'musicas', exist_ok = True )

# Set para evitar arquivos duplicados
salvos = set()

# Processar cada música
for _, row in df.iterrows():  #Ignora o índice da linha (apenas o conteúdo dela é importante)
    nome_original = row[ 'Nome da Música' ].replace( ' - Ao Vivo', '' )
    nome_limpo = re.sub( r'[\\/*?:"<>|]', '', nome_original )
    nome_arquivo = f'musicas/{nome_limpo}.txt'

    #ignorar duplicações
    if nome_arquivo in salvos:
        continue
    salvos.add( nome_arquivo )

    # Limpeza da letra
    letra = str( row[ 'Letra da Música' ] ).replace( '\n', ' ' )
    letra = re.sub( r"[!#$%&'()*+,-./:;<=>?@\\^_`{|}~]", '', letra )

    # Escrever apenas a letra no arquivo
    with open( nome_arquivo, 'w', encoding = 'utf-8' ) as f:
        f.write( letra.strip() )


In [2]:
import os
import shutil
import random

# Defina o caminho para a pasta que contém todas as músicas
source_folder = "./musicas"

# Crie as pastas para os conjuntos de treino e teste
train_folder = os.path.join( source_folder, "train" )
test_folder = os.path.join( source_folder, "test" )

os.makedirs( train_folder, exist_ok = True )
os.makedirs( test_folder, exist_ok = True )

# Obtenha a lista de músicas. Aqui é considerado que os arquivos são .txt,
# mas você pode incluir outras extensões se necessário.
musicas = [
    f for f in os.listdir( source_folder )
    if os.path.isfile( os.path.join( source_folder, f ) ) and f.lower().endswith( '.txt' )
]

# Embaralhe os arquivos para garantir a aleatoriedade
random.shuffle( musicas )

# Divida a lista de músicas em 80% para treino e 20% para teste.
split_index = int( 0.8 * len( musicas ) )
musicas_train = musicas[ :split_index ]
musicas_test = musicas[ split_index: ]

# Mova os arquivos para as respectivas pastas.
for musica in musicas_train:
    src = os.path.join( source_folder, musica )
    dst = os.path.join( train_folder, musica )
    shutil.move( src, dst )

for musica in musicas_test:
    src = os.path.join( source_folder, musica )
    dst = os.path.join( test_folder, musica )
    shutil.move( src, dst )

print( f"Total de músicas: {len( musicas )}" )
print( f"Músicas movidas para treino: {len( musicas_train )}" )
print( f"Músicas movidas para teste: {len( musicas_test )}" )


Total de músicas: 97150
Músicas movidas para treino: 77720
Músicas movidas para teste: 19430
